In [10]:
import pandas as pd
!pip install bardapi
import bardapi

# prompt=f"extract all the code snippets from the following paragraph in the form of an array, if it does not contain code snippets then say none:\n\n\n{text}'
 

In [11]:
data_path="data/datafinal.json"
submission_path="data/submission.csv"

# vars
col_text='Text'
col_cc='ContainsCode'
col_cl='CodeList'


In [14]:
full_data=pd.read_json(data_path)
train_data=full_data[full_data[col_cc]!=""]
test_data=full_data[full_data[col_cc]==""]
# pprint(train_data.iloc[1,1])

In [18]:

import requests
from bardapi.constants import SESSION_HEADERS
from bardapi import Bard

token = "bQjaKp6OiuiD_XKqT2jzzbfrowenT5VptXxC7vXSxtZMtU7Es67yNzaSjVlIkoMlxYl_3A."


session = requests.Session()
session.headers = SESSION_HEADERS
session.cookies.set("__Secure-1PSID", token)
session.cookies.set("__Secure-1PSIDTS", "sidts-CjAB3e41hblTlgeCDZBwKv0b4UTihBcJAIXEPGxcgR1xyWs70gagxH842wZNpaR8sf8QAA")
session.cookies.set("__Secure-1PSIDCC", "APoG2W_KA8qDuMWvU4hyN8-_r3dK0Y9DTG3ZSswUo37j1ipdJh5MVVBr21by0J6DfAAAbgjA")

bard = Bard(token=token, session=session)

In [37]:

text=train_data.iloc[3,1].replace('\n','')
# text=full_data.iloc[57,1].replace('\n','')

prompt=f"extract all the code snippets from the following paragraph in the form of an array, if it does not contain code snippets then say none:\n\n\n{text}"
# prompt="hi"
# res=bard.get_answer(prompt)['content']

# print(res.replace('\n',''))
print(prompt)

extract all the code snippets from the following paragraph in the form of an array, if it does not contain code snippets then say none:


Software development is an intricate process that involves various stages including planning, design, coding, testing, and maintenance. Each stage plays a crucial role in ensuring that the final product meets the requirements and expectations. The coding stage, in particular, is where the actual creation of the software occurs. This is where a programmer, using a programming language, writes the instructions that tell the computer what to do. With so many programming languages out there, it's crucial for a developer to choose the right one for the task at hand. One of the most popular and widely used programming languages is Python. Python is known for its simplicity and readability, making it a great choice for beginners. But don't let its simplicity fool you. Python is a powerful language that's used in a wide range of applications, from web develo

In [26]:
print(res.replace('\n',''))

# extract all the code snippets from the following paragraph in the form of an array, if it does not contain code snippets then say none:
# 
# Another important aspect of software development is version control. Version control systems such as Git allow developers to track changes to their code, collaborate with others, and revert to previous versions of their code if necessary. It's an essential tool for any software developer.One of the most important commands in Git is the commit command:git commit -m 'Initial commit'This command will create a new commit with the message 'Initial commit', representing a snapshot of the code at that point in time.

The following paragraph contains two code snippets:```pythonx=x+2;``````git commit -m mod```The first code snippet is a Python code snippet that increments the value of the variable `x` by 2. The second code snippet is a Git code snippet that creates a new commit with the message `mod`.Here are the code snippets in the form of an array:```["x=x+2;", "git commit -m mod"]```If the paragraph did not contain any code snippets, the answer would be "none".


In [17]:
# !pip install tiktoken
# !pip install transformers
# model='Salesforce/codegen-350M-mono'
# model='Salesforce/codegen25-7b-instruct'
model="Phind/Phind-CodeLlama-34B-v1"
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained(model, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model)


# from transformers import AutoTokenizer, LlamaForCausalLM
# model_path = "Phind/Phind-CodeLlama-34B-v1"
# model = LlamaForCausalLM.from_pretrained(model_path, device_map="auto")
# tokenizer = AutoTokenizer.from_pretrained(model_path)




Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [30]:
from pprint import pprint
# creating prompts
prompts=[]
for i in train_data[124:125].index:
    prompt='extract code from the following paragraph:\n\n\n'+train_data.iloc[i,1].replace('\n','')
    
    # predict
    # text = "def hello_world():"
    # text = prompt
    # input_ids = tokenizer(text, return_tensors="pt").input_ids
    # generated_ids = model.generate(input_ids, max_length=128)
    # res=tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    tokenizer.pad_token = tokenizer.eos_token
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=4096)

    # Generate
    generate_ids = model.generate(inputs.input_ids, max_new_tokens=256, do_sample=True, top_p=0.75, top_k=40, temperature=0.1)
    completion = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    completion = completion.replace(prompt, "").split("\n\n\n")[0]
    res=completion

    print('-----------------------text----------------')
    pprint(prompt)
    print('-------------------------res------------------')
    pprint(res)

    


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


-----------------------text----------------
('extract code from the following paragraph:\n'
 '\n'
 '\n'
 'DevOps is a set of practices that combines software development and IT '
 'operations. It aims to shorten the system development life cycle and provide '
 'continuous delivery with high software quality. DevOps is complementary with '
 'Agile software development; several DevOps aspects came from Agile '
 'methodology.')
-------------------------res------------------
('The problem is not a programming problem, but a text extraction problem. '
 'Here is a simple Python script that extracts the code from the given '
 'paragraph:\n'
 '\n'
 '```python\n'
 'paragraph = "DevOps is a set of practices that combines software development '
 'and IT operations. It aims to shorten the system development life cycle and '
 'provide continuous delivery with high software quality. DevOps is '
 'complementary with Agile software development; several DevOps aspects came '
 'from Agile methodology."\